<center>
In God We Trust
</center>

# CE417: Artificial Intelligence

Dr. Mahdiyeh Soleymani Baghshah

Computer Engineering Department,
Sharif University of Technology,
Tehran, Tehran, Iran


## Adversarial Search (25 Points)

HW3 :: Practical Question 1

### Benyamin Maleki
<br>

Corresponding TA: Aryan Ahadinia, Saeed Hedayatiyan




Consider the given $8\times8$ board game. There are two players in this game and they try to block each other and limit their valid moves by making walls. Their moves are limited to the 8 cells (or less when they are around the edges) beside them and the walls will start to appear only when a player steps into a random generated yellow cell and the wall will continue to be built until the player reaches the next yellow cell. The players start the game at $(0, 0)$ and $(7, 7)$ and they keep playing until one of them loses. There are multiple ways for a player to win the match:

- If a player steps into a wall or out of the board, the player loses the match!
- If the players reach the maximum moves the game finishes and the winner would be chosen based on the maximum valid move they have left plus the number of walls they have made and "No more moves" massege will appear on terminal.
- If the getMove() function does not return any value after 2 seconds, the player loses the match and "Time limit exceeded" massage will appear on terminal.

<center>
<img src="assets/Q3.png" style="width: 50%"/>
</center>


In this question, the codes for starting the game were provided for you in Helper-codes folder. You are expected to implement the MinimaxPlayer class (which inherit the Player class) with alpha-beta pruning.

In [ ]:
%pip install tk

You can run cell below and watch two naive players.

In [ ]:

import numpy as np
import random
import time

from Helper_codes.graphics import *
from Helper_codes.question3 import Cell
from Helper_codes.question3 import IntPair
from Helper_codes.question3 import Player
from Helper_codes.question3 import NaivePlayer
from Helper_codes.question3 import Board
from Helper_codes.question3 import Game


In [ ]:
p1 = NaivePlayer(1, 0, 0)
p2 = NaivePlayer(2, 7, 7)

g = Game(p1, p2)

numberOfMatches = 1
score1, score2 = g.start(numberOfMatches)

print(score1 / numberOfMatches)


## Below is the MinimaxPlayer 

In [23]:
import numpy as np
import random
import time

from Helper_codes.graphics import *
from Helper_codes.question3 import Cell
from Helper_codes.question3 import IntPair
from Helper_codes.question3 import Player
from Helper_codes.question3 import NaivePlayer
from Helper_codes.question3 import Board
from Helper_codes.question3 import Game


In [24]:
import math


class MinimaxPlayer(Player):
    def __init__(self, col, x, y, max_depth=4):
        super().__init__(col, x, y)
        self.max_depth = max_depth
        self.opponent = None


    possible_moves = [(1, 0), (0, 1), (-1, 0), (0, -1), (1, 1), (-1, 1), (1, -1), (-1, -1)]

    def evaluate(self, board):
        simulated_score_self = board.getScore(self.getCol())
        simulated_score_opponent = board.getScore(self.opponent.getCol())
        score = simulated_score_self - simulated_score_opponent
        return score
        
    def is_valid_move(self, result):
        if result == -2: # no more moves for player       
            return False
        if result == -1: # invalid move
            return False
        # otherwise...
        return True
  
    def min_value(self, board, alpha, beta, depth): # Opponent's turn
        # Terminal State
        if depth == 0:
            value = self.evaluate(board)
            return value, None
        # Non-terminal State
        best_successor_value = math.inf
        best_successor_move = None
        # Evaluate all possible moves
        random.shuffle(MinimaxPlayer.possible_moves)
        for move in MinimaxPlayer.possible_moves:
            simulated_board = Board(board)
            next_place = IntPair(simulated_board.getPlayerX(self.opponent.getCol()) + move[0], simulated_board.getPlayerY(self.opponent.getCol()) + move[1])
            result = simulated_board.move(next_place, self.opponent.getCol())
            # if move doesn't lead to a valid destination, skip
            if self.is_valid_move(result):
                # get successors minimax value
                value, next_plie_move = self.max_value(simulated_board, alpha, beta, depth - 1)
                # update best successor value and move
                if value < best_successor_value:
                    best_successor_value = value
                    best_successor_move = move
                # alpha-beta pruning
                beta = min(beta, best_successor_value)
                if beta <= alpha:
                    break
        return best_successor_value, best_successor_move

    def max_value(self, board, alpha, beta, depth): # Our turn
        # Terminal State
        if depth == 0:
            value = self.evaluate(board)
            return value, None
        # Non-terminal State
        best_successor_value = -math.inf
        best_successor_move = (0, 1)
        # Evaluate all possible moves
        for move in MinimaxPlayer.possible_moves:
            simulated_board = Board(board)
            next_place = IntPair(simulated_board.getPlayerX(self.getCol()) + move[0], simulated_board.getPlayerY(self.getCol()) + move[1])
            result = simulated_board.move(next_place, self.getCol())
            # if move doesn't lead to a valid destination, skip
            if self.is_valid_move(result):
                # get successors minimax value
                value, next_plie_move = self.min_value(simulated_board, alpha, beta, depth - 1)
                # update best successor value and move
                if value > best_successor_value:
                    best_successor_value = value
                    best_successor_move = move
                # alpha-beta pruning
                alpha = max(alpha, best_successor_value)
                if beta <= alpha:
                    break
        return best_successor_value, best_successor_move

    def alpha_beta_pruning(self, board):
        self.opponent = board.players[2 - self.getCol()] 
        return self.max_value(board, -np.inf, np.inf, self.max_depth)

    def getMove(self, board):
        value, first_plie_move = self.alpha_beta_pruning(board)
        return IntPair(board.getPlayerX(self.getCol()) + first_plie_move[0], board.getPlayerY(self.getCol()) + first_plie_move[1])

    


In [25]:
import traceback


p1 = MinimaxPlayer(1, 0, 0)
p2 = NaivePlayer(2, 7, 7)

g = Game(p1, p2)

number_of_matches = 1
try:
    score1, score2 = g.start(number_of_matches)
    print(score1 / number_of_matches)
except Exception as e:
    print(traceback.format_exc())
    exit()




Player 2 has exceeded the time limit
Player 1 has won

1.0
